#### Importing the Necessary Libraries.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as graph
from sklearn.pipeline import Pipeline

#### This file is the preprocessed csv file formed after applying the EDA Techniques of my first notebook. For that refer to the 'Weather.ipynb' notebook. Uploaded on both Google Drive and Slack as well.

In [ ]:
#dataset = pd.read_csv("E:/Downloads/updated_train_data.csv")E:\Omdena\Data
#dataset = pd.read_csv("train_data.csv")
dataset = pd.read_csv("E:/Omdena/Data/updated_train_data_for_WeatherII.csv")
dataset.shape

In [ ]:
dataset.sample(5)       # Getting a sample view...

#### The count of the data (especially the first two digits) is exactly similar to that of the number of days in each month. Thus, we have almost same number of data and can use similar weights for training a neural network with respect to months. We do not need to give any specific priority to the months column (its only relevance is that of the time series).

    The data points are evenly well distributed with time and date.

In [ ]:
dataset['complete'] = [1] * 375734
month_file = dataset.groupby("Month")[['complete']]     # Thus was have entire data and no entry is empty...
print(month_file.count())

In [ ]:
dataset = dataset.drop(columns="complete")
dataset.sample(5)

#### Now we will check for trends of various columns. Firstly, we will have a look at the mjo1d data and the mei data, since both of them are ratios, so if we could exclude one of them.

In [ ]:
phases = dataset[['startdate', 'mjo1d__phase', 'mjo1d__amplitude']]
fig = px.line(phases, x="startdate", y=phases.columns, title="MJO Phase and Amplitude Time Series Graph and Regression Line")
fig.update_layout(font_family="Courier New", font_color="purple", title_font_family="Times New Roman", title_font_size=24, title_font_color="red", legend_title_font_color="brown", font_size=20, legend_font_size=12)
fig.show()

#### Since it is clear the Regression lines are intersecting at some point if we traverse the graph backwards. So to get a more clear Relation, we will scale both of the features to MinMax [0,1].

In [ ]:
from sklearn.preprocessing import MinMaxScaler      # Scaler used for scaling...
'''There are many types of Scalers available like MinMax, Standard, etc.'''
MinMax = MinMaxScaler()
phases['mjo1d__phase'] = MinMax.fit_transform(np.array(phases['mjo1d__phase']).reshape(-1, 1))
phases['mjo1d__amplitude'] = MinMax.fit_transform(np.array(phases['mjo1d__amplitude']).reshape(-1, 1))

In [ ]:
fig = px.line(phases, x="startdate", y=phases.columns, title="MJO Phase and Amplitude Time Series Graph and Regression Line MinMax Scaled")     # Creating a line plot...
fig.update_layout(font_family="Courier New", font_color="purple", title_font_family="Times New Roman", title_font_size=24, title_font_color="red", legend_title_font_color="brown", font_size=20, legend_font_size=12)
fig.show()

#### Now we similarly check for the MEI Attributes.

Since the ranges of values are too far apart we cannot make a clear conclusion of the variables so we will scale them down to a common range. The Best scaler for this purpose in MinMaxScaler, since none of the below variables have negative values.

In [ ]:
phases1 = dataset[['startdate', 'mei__mei', 'mei__meirank', 'mei__nip']]
fig = px.line(phases1, x="startdate", y=phases1.columns, title="MEI Attributes Time Series Graph and Regression Line")
fig.update_layout(font_family="Courier New", font_color="purple", title_font_family="Times New Roman", title_font_size=24, title_font_color="red", legend_title_font_color="brown", font_size=20, legend_font_size=12)
fig.show()

In [ ]:
phases1['mei__mei'] = MinMax.fit_transform(np.array(phases1['mei__mei']).reshape(-1, 1))
phases1['mei__meirank'] = MinMax.fit_transform(np.array(phases1['mei__meirank']).reshape(-1, 1))
phases1['mei__nip'] = MinMax.fit_transform(np.array(phases1['mei__nip']).reshape(-1, 1))

In [ ]:
fig = px.line(phases1, x="startdate", y=phases1.columns, title="MEI Attributes Time Series Graph and Regression Line MinMax Scaled")
fig.update_layout(font_family="Courier New", font_color="purple", title_font_family="Times New Roman", title_font_size=24, title_font_color="red", legend_title_font_color="brown", font_size=20, legend_font_size=12)
fig.show()

#### From the above graph it is clear that the mei_nip has only binary values (either 0 or 1)

#### Now after performing the MinMax Scaling we see that the MJO Attributes are not correlated.

In [ ]:
def CorrelationMatrix():
    plt.figure(figsize=(7,5))
    plt.title("Correlation of MJO and MEI Ratio", color="red", size=16)
    sns.heatmap(phases.corr(), cmap="magma", annot=True)
    plt.show()

phases['Temperature'] = dataset['contest-tmp2m-14d__tmp2m']
phases['Mei_Mei'] = phases1['mei__mei']
phases['Mei_Nip'] = phases1['mei__nip']
phases['Mei_Rank'] = phases1['mei__meirank']

In [ ]:
CorrelationMatrix()

#### Intuitions from the HeatMap (After MinMax Scaling of Values) :-
1. The MJO Attributes are almost uncorrelated to each other, but they have same significance (correlation) to that of Temperature (-0.13 and -0.12). So we can eradicate one of them since they amount to same weights in evaluating Temperature.
2. Also, all the MEI Attributes are well correlated to one another (0.93, 0.81 and 1). In terms of relation to temperature, we have (0.062, 0.2, 0.082) respectively. Thus we only keep 0.2 as the column suitable for computation.
3. Since these values are MinMax scaled, applying weights for Deep Learning (SGD - Stochastic Gradient Descent) will work more effectively and quickly.

In [ ]:
class DroppingColumns(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):     # Dropping the Irrelevant columns...
        X = X.drop(columns="mei__meirank", axis=1)
        X = X.drop(columns="mei__mei", axis=1)
        X = X.drop(columns="mjo1d__amplitude", axis=1)
        return X

In [ ]:
class MinMaxScaling(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):     # Scaling down the features...
        Scaler = MinMaxScaler()
        X['mjo1d__phase'] = Scaler.fit_transform(np.array(X['mjo1d__phase']).reshape(-1, 1))
        X['mei__nip'] = Scaler.fit_transform(np.array(X['mei__nip']).reshape(-1, 1))
        return self
    
    def transform(self, X):
        return X     # Since we are not transforming the dataset, the transform function is kept empty...

In [ ]:
Pipe = Pipeline([      # Using Pipeline to call the specific classes...
    ("Remove", DroppingColumns()),
    ("MinMax", MinMaxScaling())
])
dataset = Pipe.fit_transform(dataset)
dataset.shape

#### The number of Features are reduced from 45 to 42.

#### The Seasons in USA are clustered in group of three months each. Though they do not comprise the entirety of each month, these are the meteorological seasons:
1. Spring months: March, April, May
2. Summer months: June, July, August
3. Fall months: September, October, November
4. Winter months: December, January, February

#### Now we will do an Analysis for Spring Season (March, April and May).
    Since the || and && are ambiguous for dataframe, we use bitwise | and & which are overloaded operators for pandas.

In [ ]:
Spring = dataset[(dataset['Month'] == 3) | (dataset['Month'] == 4) | (dataset['Month'] == 5)]

In [ ]:
class RenamingWinds(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X['Height10'] = X['contest-wind-h10-14d__wind-hgt-10']
        X['Height100'] = X['contest-wind-h100-14d__wind-hgt-100']
        X['Height500'] = X['contest-wind-h500-14d__wind-hgt-500']
        X['Height850'] = X['contest-wind-h850-14d__wind-hgt-850']
        X['LongitudinalWind925'] = X['contest-wind-vwnd-925-14d__wind-vwnd-925']
        X['LongitudinalWind250'] = X['contest-wind-vwnd-250-14d__wind-vwnd-250']
        X['ZonalWind250'] = X['contest-wind-uwnd-250-14d__wind-uwnd-250']
        X['ZonalWind925'] = X['contest-wind-uwnd-925-14d__wind-uwnd-925']
        X = X.drop(['contest-wind-h10-14d__wind-hgt-10', 'contest-wind-h100-14d__wind-hgt-100', 'contest-wind-h500-14d__wind-hgt-500', 'contest-wind-h850-14d__wind-hgt-850', 'contest-wind-vwnd-925-14d__wind-vwnd-925', 'contest-wind-vwnd-250-14d__wind-vwnd-250', 'contest-wind-uwnd-250-14d__wind-uwnd-250', 'contest-wind-uwnd-925-14d__wind-uwnd-925'], axis=1)
        return X

In [ ]:
class RenamingTemperatures(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X['Mean0Temperature34w'] = X['nmme0-tmp2m-34w__nmme0mean']    # Most Recent Mean...
        X['Temperature'] = X['contest-tmp2m-14d__tmp2m']
        X['MeanTemperature34w'] = X['nmme-tmp2m-34w__nmmemean']
        X['Nmme0Mean'] = X['nmme0mean']        # NMME Mean...
        X['Evaporation'] = X['contest-pevpr-sfc-gauss-14d__pevpr']
        X['RecentCancmPrate56w'] = X['nmme0-prate-56w__cancm40']    # Cancm40 is taken...
        X['RecentCcsmPrate56w'] = X['nmme0-prate-56w__ccsm30']     # Ccsm30 is taken...
        X['CancmPrate56w'] = X['nmme-prate-56w__cancm4']              # Cancm4...
        X['Ccsm3Prate56w'] = X['nmme-prate-56w__ccsm3']                # Ccsm3...
        X = X.drop(['nmme0-tmp2m-34w__nmme0mean', 'contest-tmp2m-14d__tmp2m', 'nmme0mean', 'nmme-tmp2m-34w__nmmemean', 'contest-pevpr-sfc-gauss-14d__pevpr', 'nmme0-prate-56w__cancm40', 'nmme0-prate-56w__ccsm30', 'nmme-prate-56w__cancm4', 'nmme-prate-56w__ccsm3'], axis=1)
        return X

In [ ]:
class RenamingPrecipitation(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X['Humidity'] = X['contest-rhum-sig995-14d__rhum']
        X['SeaLevelPressure'] = X['contest-slp-14d__slp']
        X['Pressure'] = X['contest-pres-sfc-gauss-14d__pres']
        X['AtmosphericPrecipitate'] = X['contest-prwtr-eatm-14d__prwtr']
        X['Precipitation'] = X['contest-precip-14d__precip']
        X['Elevation'] = X['elevation__elevation']
        X['Regions'] = X['climateregions__climateregion']
        X = X.drop(['contest-rhum-sig995-14d__rhum', 'contest-slp-14d__slp', 'contest-pres-sfc-gauss-14d__pres', 'contest-precip-14d__precip', 'elevation__elevation', 'climateregions__climateregion', 'contest-prwtr-eatm-14d__prwtr'], axis=1)
        return X

In [ ]:
class ElNinoFactors(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X['ElNinoLongitudinalWind250Mean'] = X['wind-vwnd-250-2010-mean']
        X['ElNinoZonalWind250Mean'] = X['wind-uwnd-250-2010-mean']
        X['ElNinoZonalWind925Mean'] = X['wind-uwnd-925-2010-mean']
        X['ElNinoLongitudinalWind925Mean'] = X['wind-vwnd-925-2010-mean']
        X['GlacierFactorMean'] = X['icec-2010-mean']
        X['ElNinoHeight10Mean'] = X['wind-hgt-10-2010-mean']
        X['ElNinoHeight100Mean'] = X['wind-hgt-100-2010-mean']
        X['ElNinoHeight500Mean'] = X['wind-hgt-500-2010-mean']
        X['ElNinoHeight850Mean'] = X['wind-hgt-850-2010-mean']
        X['ElNinoSeaTemperatureMean'] = X['sst-2010-mean']
        X = X.drop(['wind-vwnd-250-2010-mean', 'wind-uwnd-925-2010-mean', 'wind-uwnd-250-2010-mean', 'wind-vwnd-925-2010-mean', 'icec-2010-mean', 'wind-hgt-850-2010-mean', 'wind-hgt-500-2010-mean', 'wind-hgt-100-2010-mean', 'wind-hgt-10-2010-mean', 'sst-2010-mean'], axis=1)
        return X

    This Pipeline is just used for naming of columns to make the understanding of the variables a bit more clear. Otherwise, it was quite difficult to understand the general nomenclature of the columns.

In [ ]:
ColumnNameTransformationPipe = Pipeline([
    ("RenameI", RenamingWinds()),
    ("RenameII", RenamingTemperatures()),
    ("RenameIII", RenamingPrecipitation()),
    ("RenameIV", ElNinoFactors())
])
SpringSeason = ColumnNameTransformationPipe.fit_transform(Spring)

In [ ]:
SpringSeason.shape

#### Now we will check the Environmental Factors Correlation To Temperature in Spring Season via HeatMap.

In [ ]:
var = SpringSeason[['Temperature', 'Height10', 'Height100', 'Height500', 'Height850', 'LongitudinalWind925', 'LongitudinalWind250', 'ZonalWind250', 'ZonalWind925', 'Mean0Temperature34w' , 'MeanTemperature34w', 'Nmme0Mean', 'Elevation']]

def Plotting(variable):
    plt.figure(figsize=(10, 7.5))
    plt.title("Correlation in Spring Season", color="red", size=24)
    sns.heatmap(variable.corr(), cmap="icefire", annot=True)
    plt.show()

Plotting(var)

#### Intuitions from the Correlation Matrix-
1. Geo-potential Height depends upon the Temperature and Pressure of the region but it does not depend upon the Elevation.
2. Mean0Temperature34w (recent) is highly correlated to MeanTemperature34w (0.97) and similarly for Nmme0Mean (0.93). Thus we will MinMax scale them and check their correlation again.
3. Height100 and Height500 are well correlated (0.94) so we can exclude one of them.
4. Greater the Elevation, lower will be the temperature.

In [ ]:
plot = SpringSeason[['Elevation', 'Regions', 'startdate', 'Temperature']]
fig = px.histogram(plot, x="Regions", title="Data Points Count in Spring Season")           # The Cold Semi-Arid Climate is the most common one...
fig.update_layout(title_font_color="red", title_font_size=24, font_color="purple", font_size=18)
fig.show()

#### Intuition-
1. Since the Cold Semi-Arid Region data prevails in the Spring Season, there must be a wide thermal variation between day and night, lower amount of precipitation than evaporation and shortage of humidity (the wind is mostly dry). These factors are the most prevailing factors in the Cold Semi-Arid Regions.
2. The Semi-Arid data points amount to more than 50% of total data points in the Spring Season. Thus, while training our Deep learning or Machine Learning Model the SGD (Stochastic Gradient Descent) can probably have a faster learning rate influenced by the BSk data points.

In [ ]:
fig = px.histogram(SpringSeason, x="startdate", y="Temperature", histnorm='probability density', title="Probability Density of Spring Season of Two Years (Showing a generalized Increasing Trend)")
fig.update_layout(bargap=0.1, title_font_size=24, title_font_color="red", font_size=16, font_color="purple")
fig.show()

#### Intuition-
1. Since the Probability of both years shows a generalized increase in Temperature Probability. It very well follows the generalized Normal Distribution curve.
2. Thus we can Normally scale (Standard Scaler) to scale down the temperature variables and the variables closely affected by the Temperature (Normal Distribution:- Mean:0, Variance:1). Thus, we can scale down all the Temperature variables by Standard Scaling.

In [ ]:
var1 = SpringSeason[['Temperature', 'Evaporation' ,'RecentCancmPrate56w', 'RecentCcsmPrate56w', 'CancmPrate56w', 'Ccsm3Prate56w', 'Humidity', 'SeaLevelPressure', 'Pressure', 'AtmosphericPrecipitate', 'Precipitation', 'Elevation']]
Plotting(var1)

#### Intuition -
1. The Pressure and Elevation are highly negatively correlated ( -0.92 ) and they have same correlation or rather the almost the same affect on Temperature ( -0.47 and -0.41 ), so we can eradicate any one of them.
2. Since other factors are not well correlated to any of them, we should not remove any one of them.

Now we will check for ElNino Factors.

In [ ]:
var2 = SpringSeason[['Temperature', 'ElNinoLongitudinalWind250Mean', 'ElNinoZonalWind250Mean', 'ElNinoZonalWind925Mean', 'ElNinoLongitudinalWind925Mean', 'GlacierFactorMean', 'ElNinoHeight10Mean', 'ElNinoHeight100Mean', 'ElNinoHeight500Mean', 'ElNinoHeight850Mean', 'ElNinoSeaTemperatureMean']]
Plotting(var2)

#### Intuition -
1. The ElNinoHeight columns are very closely related to each other thus, we will only take one of them and remove the rest, we will keep ElNinoHeight10Mean and ElNinoHeight850Mean.
2. The Longitudinal and Zonal Wind 250 have the almost same correlation values with Temperature (0.38 and 0.39) such that they have the same degree of influence on the Temperature, thus we will remove the which is more correlated to the other variables (ElNinoZonalWind250Mean)
3. Also the ZonalWinds are more correlated to Goe-potential Height than the LongitudinalWinds, which means, the Wind flowing in Horizontal Direction poses more impact on other features than the Vertical wind.

In [ ]:
class RemovingColumns(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.drop(['MeanTemperature34w', 'Nmme0Mean', 'Height500', 'Pressure', 'ElNinoHeight100Mean', 'ElNinoHeight500Mean', 'ElNinoZonalWind250Mean'], axis=1)
        return X

In [ ]:
FilteringPipe = Pipeline([
    ("Removing", RemovingColumns())
])
SpringSeason = FilteringPipe.fit_transform(SpringSeason)
SpringSeason.shape

#### The number of Columns have been reduced from 42 to 35.

    Now we will remove the Time series data to have a more general correlation view of various parameters for the Spring Season.

In [ ]:
class TimeDataExclusion(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):     # We removed the time series data...
        X = X.drop(columns="startdate", axis=1)
        X = X.drop(columns="date", axis=1)
        X = X.drop(['Month', 'Year', 'lat', 'lon'], axis=1)
        return X

Since Latitudes and Longitudes are constant with and not variable with Time Series data, they can be simply passed along with the weights, to ascertain the coordinates. Just a simple reminder to Geography, the latitudes and longitudes do not influence any environmental factors, they are just to represent the coordinates on a Map or a Globe. So they can be included with loss function or the metrics while training the Deep Learning Model.

In [ ]:
ExclusionPipe = Pipeline([
    ("Exclude", TimeDataExclusion())
])
SpringSeason1 = ExclusionPipe.transform(SpringSeason)
SpringSeason1.shape

    Now the controlling factors have been reduced from 35 to 29.

Now we will create the correlation matrix of the entire factors as whole to see their influence on Temperature during the Spring Season.

In [ ]:
plt.figure(figsize=(16, 13))
plt.title("Correlation Of Various Factors In Spring Season", color="blue", size=24)
sns.heatmap(SpringSeason1.corr(), cmap="Spectral", annot=True)
plt.show()

#### Since the data is time series data, the correlation values are already in the same time frame such that the values are calculated by taking the time into consideration since, the data is already sorted in a continous flow according to time.

#### Intuition -
1. SeaLevelPressure and GlacierFactorMean have same correlation with respect to Temperature (-0.47 and -0.46).
2. Height850 and ElNinoSeaTemperatureMean have pretty similar correlation with respect to Temperature (0.42 and 0.43). This means that the Geo-potential Height at 850 millibar and ElNinoSeaTemperature has the same effect to the Temperature of the regions, thus the Regions of Height850 are pretty well influenced by the ElNinoSeaTemperatureMean.
3. ElNinoHeight10Mean is very negatively correlated to Height10 (-0.89) and they have same magnitude of effect on Temperature (0.52 and -0.52). Thus, we can exclude one of them since they have same degree of effect and are negatively correlated to each other.
4. MJO Ratio is not well related to any factor and neither the Temperature as well but the MEI Ratio has an enhanced relation to other factors and the Temperature as compared to that of MJO Ratio. Thus, from the heatmap it would be preferred that we take MEI Ratio and exclude MJO Ratio.
5. Height850 and AtmosphericPrecipitate have same correlation with Height100 (0.61). Height100 is fairly well correlated to Temperature (0.79). So, since the two factors have same correlation values (0.61) to a variable we can exclude one of them from the Computation.
6. LongitudinalWind925 and LongitudinalWind250 have same correlation with the Mean0Temperature34w (0.34), so we can exclude one of them and it also gives an idea, that the LongitudinalWinds provided have the similar properties (not exactly same).
7. Humidity, RecentCcsmPrate56w and SeaLevelPressure have same effect on the Height100 (-0.32). This means, that for Spring Season our chosen weather station gives the correct 56 week precipitate and has the same influence on the Height100 as that of the Humidity and SeaLevelPressure (the two main factors on which the Precipitation depends).
8. The ZonalWind925 and Elevation have the same correlation values with respect to Mean0Temperature34w (-0.35). This gives us a glimpse of idea that at the Zonal Winds and the Elevation has the same effect on the Computation of Temperature by the weather forecasts.
9. LongitudinalWind925 and ElNinoZonalWind925 have exactly same correlation values on the Temperature (0.38). Thus, there is a high chance that these two winds have the same characteristics and play the same role in determining the temperature.

#### The columns which we will further remove:-
    GlacierFactorMean, ElNinoSeaTemperatureMean, ElNinoHeight10Mean, mjo1d__phase, AtmosphericPrecipitate, LongitudinalWind925, SeaLevelPressure, Humidity, ZonalWind925, ElNinoZonalWind925, etc.

In [ ]:
class RemovingColumns(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):      # Pipeline created to remove the further irrelevant columns...
        X = X.drop(columns="GlacierFactorMean", axis=1)
        X = X.drop(columns="ElNinoSeaTemperatureMean", axis=1)
        X = X.drop(columns="ElNinoHeight10Mean", axis=1)
        X = X.drop(columns="mjo1d__phase", axis=1)
        X = X.drop(columns="AtmosphericPrecipitate", axis=1)
        X = X.drop(columns="LongitudinalWind925", axis=1)
        X = X.drop(columns="SeaLevelPressure", axis=1)
        X = X.drop(columns="Humidity", axis=1)
        X = X.drop(columns="ElNinoZonalWind925Mean", axis=1)
        X = X.drop(columns="ZonalWind925", axis=1)
        X = X.drop(columns="RecentCancmPrate56w", axis=1)
        return X

In [ ]:
ColumnRemovePipe = Pipeline([
    ("Removing", RemovingColumns())
])
SpringSeason1 = ColumnRemovePipe.transform(SpringSeason1)
SpringSeason1.shape

#### The number of Significant columns have been reduced from 29 to 18.

    The below graph shows the general temperature trend. Take a look of a year and we will find a bell shaped curve similar to that of the Normal Distribution curve. The little noise in the left domain can easily be rectified by including a squared bias function in the Deep Learning Model while training.

In [ ]:
fig = px.histogram(dataset, x="startdate", y="contest-tmp2m-14d__tmp2m", histnorm='probability density', title="Probability Temperature Density of All Season of Two Years (A Generalized Normal Distribution)")
fig.update_layout(bargap=0.1, title_font_size=24, title_font_color="red", font_size=16, font_color="purple", height=600)
fig.show()

In [ ]:
fig = px.histogram(dataset, x="startdate", y="nmme0-tmp2m-34w__nmme0mean", histnorm='probability density', title="Probability Temperature Density of Chosen Station All Season of Two Years (A Generalized Normal Distribution)")
fig.update_layout(bargap=0.1, title_font_size=24, title_font_color="red", font_size=16, font_color="purple", height=600)
fig.show()

    Looking at the above two graphs this gives us an intuition that if we scale them down using Standard Scaler, both of them will become more continuous and correlated with each other. Even before scaling them show a very clear positive correlation of 0.92.

In [ ]:
SpringSeason1['latitude'] = SpringSeason['lat']     # Latitudes added...
SpringSeason1['longitude'] = SpringSeason['lon']    # Longitudes added...
plt.figure(figsize=(15, 13))
plt.title("Final HeatMap of Factors Of Spring Season", color="red", size=22)
sns.heatmap(SpringSeason1.corr(), cmap="magma", annot=True)
plt.show()

#### Intuitions:-
1. Latitude is significantly negatively correlated to the Temperature and Evaporation Parameters.
2. Longitude is negatively correlated to Recent Data of Ccsm station.
3. Important:- Almost every factor has a 20% effect on Temperature calculation.

In [ ]:
SpringSeason1['Startdate'] = SpringSeason['startdate']    # Start date and regions also added...
SpringSeason1['regions'] = SpringSeason['Regions']
SpringSeason1.shape

In [ ]:
fig = px.density_contour(SpringSeason1, x="Mean0Temperature34w", y="Height100", title="Temperature0 X Height100")
fig.update_traces(contours_coloring="fill", contours_showlabels = True, colorscale="viridis")
fig.update_layout(width=1000, height=650, font_family="Courier New", font_color="green",     title_font_family="Times New Roman", title_font_color="red", title_font_size=24, font_size=16)
fig.show()

There is only one Intensity Point (Maxima) and is a global one, so we can use either exponential scaling or the Logarithmic Scaling (its inverse) with respect to Height100.

In [ ]:
fig = px.density_contour(SpringSeason1, x="Mean0Temperature34w", y="latitude", title="Temperature0 X Latitude")
fig.update_traces(contours_coloring="fill", contours_showlabels = True, colorscale="Thermal")
fig.update_layout(width=1000, height=650, font_family="Courier New", font_color="green",     title_font_family="Times New Roman", title_font_color="red", title_font_size=24, font_size=16)
fig.show()

There are two local Maxima and thus we can provide more bias and weight regarding the latitudes Maxima. The Maxima is pretty dense.

In [ ]:
fig = px.density_contour(SpringSeason1, x="Mean0Temperature34w", y="longitude", title="Temperature0 X Longitude")
fig.update_traces(contours_coloring="fill", contours_showlabels = True, colorscale="Thermal")
fig.update_layout(width=1000, height=650, font_family="Courier New", font_color="green",     title_font_family="Times New Roman", title_font_color="red", title_font_size=24, font_size=16)
fig.show()

Similarly, we have two local Maxima for Longitudes and then weigh them accordingly as per the Longitude. Also, here in this case, we have heavy Maxima (heavy Intensity point).

    The Intensity of Maxima determines the learning rate of the Deep Learning Algorithm especially for the SGD and Regression.